In [1]:
#Libraries
import pandas as pd
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from varclushi import VarClusHi
from collections import Counter

In [2]:
#Reading the csv file into a dataframe
loanData = pd.read_csv('/Users/vibhor/Desktop/Models/LC/Loan_status_2007-2020Q3.gzip', index_col=0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (1,48,58,117,127,128,129,132,133,134,137) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df = loanData

In [ ]:
loanData.duplicated().sum() # no duplicates

In [ ]:
#Dimensions of the dataframe
loanData.shape

In [ ]:
loanData.info()

In [ ]:
#only when needed, and delete display after it is shown
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

In [ ]:
#Columns of the dataframe, do
#loanData.columns #too many columns and so all the cols do not get listed this way
list(loanData.columns) #need to print it as a list

In [ ]:
len(loanData.columns[loanData.isna().any()]) # Except the id column all others have N
#to print them
#loanData.columns[loanData.isna().any().tolist()


In [ ]:
#target variable is loan_status. Understand what values it takes and its corresponding counts
loanData['loan_status'].value_counts()

In [3]:
missing_data = loanData.isna().mean() #by variable this returns count(na)/total entries
#missing_data.plot.hist(bins=20)

In [4]:
drop_list = list(missing_data[missing_data > 0.2].index)
print("\n\n Drop Features: ", len(drop_list))
print(drop_list)



 Drop Features:  49
['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 'orig_projected_a

Some of these variables are important but they have just too many missing values to fill, thats why dropping them all

In [ ]:
df.loc[:,drop_list].isna().sum()

In [5]:
#### From another person's code
remove_list = ['collection_recovery_fee',
               'grade',                  
               'initial_list_status',
               'next_pymnt_d',
               'policy_code',
               'title',
               'url',
               'payment_plan_start_date',
               'orig_projected_additional_accrued_interest',
               'debt_settlement_flag_date',
               'settlement_date']

In [6]:
temporal_vars_remove = ['last_credit_pull_d','last_fico_range_high','last_fico_range_low','total_pymnt',
                        'total_pymnt_inv','recoveries','collection_recovery_fee','out_prncp','out_prncp_inv',
                        'total_rec_prncp','total_rec_int','last_pymnt_d','last_pymnt_amnt','next_pymnt_d',
                        'total_rec_late_fee','hardship_flag','hardship_amount',
                        'orig_projected_additional_accrued_interest','hardship_payoff_balance_amount',
                        'hardship_last_payment_amount','debt_settlement_flag','hardship_type','hardship_reason',
                        'hardship_status','deferral_term','hardship_start_date','hardship_end_date','payment_plan_start_date',
                        'hardship_length','hardship_dpd','hardship_loan_status','installment','pymnt_plan']

In [7]:
#use Venn Diagram. Set function is not needed as there wont be any repeats in the lists, but used for completeness
#In A but not in B or C + In B but not in C + C
vars_dropped = list(set(drop_list) - set(remove_list) - set(temporal_vars_remove)) + \
list(set(remove_list) - set(temporal_vars_remove)) + temporal_vars_remove
len(vars_dropped)

74

In [8]:
retained_variables = list(set(loanData.columns) - set(vars_dropped))
len(retained_variables)

69

In [ ]:
print(retained_variables)

In [9]:
df_smaller = loanData.loc[:,retained_variables] #dataframe of retained variables. 

In [ ]:
#new COflag
problematic = ['Charged Off','Late (31-120 days)', 'Default']
df_smaller['coFlag'] = [1 if x in problematic else 0 for x in loanData['loan_status']]

In [10]:
df_intr = df_smaller['int_rate']

In [29]:
df_smaller.drop('int_rate', axis = 1, inplace = True)

In [11]:
df_intr2 = pd.DataFrame(df_intr.str.rstrip('%').astype('float'))

In [30]:
df_refined = pd.concat([df_intr2, df_smaller], axis=1)

In [31]:
temp_means = (df_refined._get_numeric_data().mean())

In [32]:
#counting number of object type (categorical) and numeric and getting a list of those
#Counter(df_refined.dtypes)
num_cols = df_refined._get_numeric_data().columns
cat_cols = list(set(df_refined.columns) - set(num_cols))

In [33]:
df_num = df_refined.loc[:, num_cols]

In [34]:
num_cols

Index(['int_rate', 'acc_now_delinq', 'percent_bc_gt_75', 'num_bc_sats',
       'delinq_2yrs', 'pct_tl_nvr_dlq', 'bc_open_to_buy', 'tot_cur_bal',
       'pub_rec_bankruptcies', 'bc_util', 'collections_12_mths_ex_med',
       'total_bal_ex_mort', 'num_rev_tl_bal_gt_0', 'dti', 'fico_range_high',
       'chargeoff_within_12_mths', 'mo_sin_rcnt_tl', 'acc_open_past_24mths',
       'mths_since_recent_inq', 'num_tl_120dpd_2m', 'num_actv_bc_tl',
       'total_acc', 'funded_amnt_inv', 'inq_last_6mths', 'mo_sin_old_il_acct',
       'tax_liens', 'avg_cur_bal', 'tot_hi_cred_lim', 'mort_acc', 'num_il_tl',
       'pub_rec', 'fico_range_low', 'num_tl_30dpd', 'num_bc_tl',
       'num_accts_ever_120_pd', 'annual_inc', 'mths_since_recent_bc',
       'mo_sin_old_rev_tl_op', 'loan_amnt', 'revol_bal', 'num_rev_accts',
       'funded_amnt', 'total_rev_hi_lim', 'num_sats',
       'total_il_high_credit_limit', 'total_bc_limit', 'mo_sin_rcnt_rev_tl_op',
       'num_tl_90g_dpd_24m', 'delinq_amnt', 'num_actv_rev_

In [35]:
for var in num_cols:
    df_num[var].fillna(temp_means[var], inplace = True)   

In [36]:
df_num.isna().any()

int_rate                      False
acc_now_delinq                False
percent_bc_gt_75              False
num_bc_sats                   False
delinq_2yrs                   False
pct_tl_nvr_dlq                False
bc_open_to_buy                False
tot_cur_bal                   False
pub_rec_bankruptcies          False
bc_util                       False
collections_12_mths_ex_med    False
total_bal_ex_mort             False
num_rev_tl_bal_gt_0           False
dti                           False
fico_range_high               False
chargeoff_within_12_mths      False
mo_sin_rcnt_tl                False
acc_open_past_24mths          False
mths_since_recent_inq         False
num_tl_120dpd_2m              False
num_actv_bc_tl                False
total_acc                     False
funded_amnt_inv               False
inq_last_6mths                False
mo_sin_old_il_acct            False
tax_liens                     False
avg_cur_bal                   False
tot_hi_cred_lim             

In [ ]:
df_num.loc[df_num['annual_inc'].isna(), listofvars]

In [ ]:
#df_num['annual_inc'].fillna(replaceVal_flag1, inplace = True)
pd.DataFrame(df_num[df_num['statusFlag']==1]['annual_inc']).fillna(replaceVal_flag1, inplace = True)

In [ ]:
df_num['annual_inc'].isna().any()

In [ ]:
pd.DataFrame(df_num[df_num['statusFlag']==1]['annual_inc']).head()

# #Clustering

In [ ]:
df_Y = df_num.pop('coFlag')

In [37]:
df_num_short = df_num.sample(frac = 0.3)

In [38]:
loanData_vc = VarClusHi(df_num_short,maxclus=None)

In [39]:
loanData_vc.varclus()

In [40]:
loanData_vc.corrs

,int_rate,acc_now_delinq,percent_bc_gt_75,num_bc_sats,delinq_2yrs,pct_tl_nvr_dlq,bc_open_to_buy,tot_cur_bal,pub_rec_bankruptcies,bc_util,...,total_il_high_credit_limit,total_bc_limit,mo_sin_rcnt_rev_tl_op,num_tl_90g_dpd_24m,delinq_amnt,num_actv_rev_tl,num_op_rev_tl,num_tl_op_past_12m,open_acc,tot_coll_amt
int_rate,1.000000,0.013960,0.269921,-0.058515,0.068071,-0.089273,-0.287997,-0.078565,0.050811,0.274959,...,0.001274,-0.234866,-0.084903,0.037277,0.004571,0.081726,-0.015161,0.163893,-0.008007,0.013735
acc_now_delinq,0.013960,1.000000,-0.010828,-0.002421,0.106414,-0.040515,0.004071,0.018016,-0.006362,-0.013455,...,0.009169,-0.000311,-0.000315,0.047461,0.173139,0.003400,0.011237,-0.003684,0.015478,0.002554
percent_bc_gt_75,0.269921,-0.010828,1.000000,-0.162723,0.006114,-0.021452,-0.440606,0.035078,-0.031518,0.846344,...,0.008406,-0.213380,0.122456,-0.003870,-0.001503,0.095147,-0.131428,-0.178898,-0.089362,-0.024909
num_bc_sats,-0.058515,-0.002421,-0.162723,1.000000,-0.031198,0.146400,0.507548,0.125996,-0.050951,-0.178543,...,0.073816,0.624775,-0.207716,-0.025210,0.001962,0.683845,0.779440,0.254387,0.655695,-0.008761
delinq_2yrs,0.068071,0.106414,0.006114,-0.031198,1.000000,-0.437718,-0.063298,0.053359,-0.046925,0.010458,...,0.047817,-0.079385,0.024630,0.649568,0.025682,0.000311,0.005120,-0.024309,0.041385,0.002269
pct_tl_nvr_dlq,-0.089273,-0.040515,-0.021452,0.146400,-0.437718,1.000000,0.168459,0.012750,0.054426,-0.039734,...,0.025356,0.216814,-0.029616,-0.256525,-0.013456,0.096113,0.135398,0.011903,0.110651,-0.051266
bc_open_to_buy,-0.287997,0.004071,-0.440606,0.507548,-0.063298,0.168459,1.000000,0.168139,-0.089370,-0.541980,...,0.097717,0.846903,-0.078912,-0.045308,0.000469,0.136815,0.380196,0.128896,0.325642,-0.019030
tot_cur_bal,-0.078565,0.018016,0.035078,0.125996,0.053359,0.012750,0.168139,1.000000,-0.082602,0.044389,...,0.439264,0.312958,0.030356,0.005744,0.016149,0.106653,0.104129,0.106720,0.257619,-0.008928
pub_rec_bankruptcies,0.050811,-0.006362,-0.031518,-0.050951,-0.046925,0.054426,-0.089370,-0.082602,1.000000,-0.026988,...,-0.035207,-0.136853,-0.080396,-0.012695,0.000144,-0.004749,0.017286,0.089708,-0.009567,0.002078
bc_util,0.274959,-0.013455,0.846344,-0.178543,0.010458,-0.039734,-0.541980,0.044389,-0.026988,1.000000,...,0.015282,-0.251778,0.125612,-0.001863,-0.001715,0.125721,-0.148527,-0.196458,-0.101274,-0.022677


In [41]:
loanData_vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,num_bc_sats,0.743137,0.275206,0.354395
1,0,num_rev_tl_bal_gt_0,0.773255,0.143379,0.264697
2,0,num_actv_bc_tl,0.687628,0.198367,0.389670
3,0,num_bc_tl,0.601054,0.223680,0.513893
4,0,num_rev_accts,0.653508,0.213029,0.440286
5,0,num_sats,0.748706,0.372941,0.400750
6,0,num_actv_rev_tl,0.776136,0.143614,0.261405
7,0,num_op_rev_tl,0.877953,0.195777,0.151757
8,0,open_acc,0.743638,0.377000,0.411496
9,1,tot_cur_bal,0.931107,0.256515,0.092662



#RF for var select
#IV and WOE
